<font size="8">What is CVXPY?</font>

This workbook is based on https://www.cvxpy.org/index.html

CVXPY is an open source Python-embedded modeling language for convex optimization problems.It automatically transforms the problem into standard form, calls a solver, and unpacks the results.

<font size="8">Example</font>

\begin{align*}
\text{minimize: } & (x-y)^2, \\
\text{subject to: } & x+y=1, \\
 & x-y \geq 1, \\
\end{align*}
where $x,y \in \mathbb{R}$

In [18]:
import cvxpy as cp
import numpy as np

# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var",'x is', x.value,'y is', y.value)

status: optimal
optimal value 1.0
optimal var x is 1.0 y is 1.570086213240983e-22


<font size="8">Status</font>

- OPTIMAL

- INFEASIBLE

- UNBOUNDED

- OPTIMAL_INACCURATE

- INFEASIBLE_INACCURATE

- UNBOUNDED_INACCURATE

- INFEASIBLE_OR_UNBOUNDED

<font size="8">Variable</font>

Variables can be scalars, vectors, or matrices, meaning they are 0, 1, or 2 dimensional.

In [19]:
# A scalar variable.
a = cp.Variable()

# Vector variable with shape (5,).
x = cp.Variable(5)

# Matrix variable with shape (5, 1).
x = cp.Variable((5, 1))

# Matrix variable with shape (4, 7).
A = cp.Variable((4, 7))

<font size="6">Currently the following types may be used as constants:</font>

- NumPy ndarrays

- NumPy matrices

- SciPy sparse matrices

<font size="8">Example</font>

\begin{align*}
\text{minimize: } & \sum_{ij}(Ax-b)^2, \\
\text{subject to: } & x \geq 0 \\
 & x \leq 1, \\
\end{align*}

In [20]:
# Solves a bounded least-squares problem.

import cvxpy as cp
import numpy

# Problem data.
m = 10
n = 5
numpy.random.seed(1)
A = numpy.random.randn(m, n)
b = numpy.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)
prob.solve()

print("status:", prob.status)
print("Optimal value", prob.value)
print("Optimal var")
print('x is ',x.value) # A numpy ndarray.

status: optimal
Optimal value 4.141338603672535
Optimal var
x is  [-4.95922264e-21  6.07571976e-21  1.34643668e-01  1.24976681e-01
 -4.57130806e-21]


<font size="8">Constraints</font>

As shown in the example code, you can use ==, <=, and >= to construct constraints in CVXPY. Equality and inequality constraints are elementwise, whether they involve scalars, vectors, or matrices. For example, together the constraints 0 <= x and x <= 1 mean that every entry of x is between 0 and 1.

You cannot construct inequalities with < and >. Strict inequalities don’t make sense in a real world setting. Also, you cannot chain constraints together, e.g., 0 <= x <= 1 or x == y == 2. The Python interpreter treats chained constraints in such a way that CVXPY cannot capture them. CVXPY will raise an exception if you write a chained constraint.

<font size="8">Operators</font>

The infix operators +, -, *, /, @ are treated as functions. 
- @ should be used for matrix-matrix and matrix-vector multiplication,

- $\text{*}$ should be matrix-scalar and vector-scalar multiplication

<font size="8">Indexing and slicing</font>

Indexing in CVXPY follows exactly the same semantics as NumPy ndarrays. For example, if expr has shape (5,) then expr[1] gives the second entry. More generally, expr[i:j:k] selects every kth element of expr, starting at i and ending at j-1. If expr is a matrix, then expr[i:j:k] selects rows, while expr[i:j:k, r:s:t] selects both rows and columns. Indexing drops dimensions while slicing preserves dimensions. For example,

In [21]:
x = cp.Variable(5)
print("0 dimensional", x[0].shape)
print("1 dimensional", x[0:1].shape)

0 dimensional ()
1 dimensional (1,)


<font size="8">Transpose</font>

The transpose of any expression can be obtained using the syntax <mark>expr.T</mark>. Transpose is an affine function.

<font size="8">Power</font>

For any CVXPY expression <mark>expr</mark>, the power operator <mark>expr**p</mark> is equivalent to the function <mark>power(expr, p)</mark>.

<font size="8">Linear Programming</font>

\begin{align*}
\text{minimize:} & \quad c^T x, \\
\text{subject to:} & \quad A_{ub} x \leq b_{ub}, \\
 & \quad A_{eq} x = b_{eq}, \\
\end{align*}
\
where

$A_{ub} \in  \mathbb{R}^{m \times n},b_{ub} \in \mathbb{R}^m,A_{eq} \in  \mathbb{R}^{s \times n},b_{eq} \in \mathbb{R}^s \text{ and } c \in \mathbb{R}^n \text{ are problem data and } x \in  \mathbb{R}^n \text{ is the optimization variable.}$

<font size="8">Example 1</font>

You have $12,000 to invest, and three different funds from which to choose. The municipal bond fund has a 7% return, the local bank's CDs have an 8% return, and the high-risk account has an expected (hoped for) 12% return. To minimize the risk, you decide not to invest any more than $2,000 in the high-risk account. For tax reasons, you need to invest at least three times as much in the municipal bonds as in the bank CDs. Assuming the year-end yields are expected what are the optimal investment amounts?

# Investment Optimization Problem

You have $12,000 to invest in three different funds: municipal bonds, local bank's CDs, and a high-risk account. The objective is to maximize the total return on investment while considering certain constraints to minimize risk and meet tax requirements.

## Objective Function
The objective is to maximize the total return:
$ \text{Maximize} \quad 0.07x_1 + 0.08x_2 + 0.12x_3 $

where:
- $x_1$: amount invested in municipal bonds
- $x_2$: amount invested in local bank's CDs
- $x_3$: amount invested in the high-risk account

## Constraints
1. **Total investment is limited to $12,000:**
   $x_1 + x_2 + x_3 \leq 12,000$

2. **The amount invested in the high-risk account is limited to $2,000:**
   $x_3 \leq 2,000$

3. **For tax reasons, invest at least three times as much in municipal bonds as in bank CDs:**
   $x_1 \geq 3x_2$

In [22]:
# Define variables
x1 = cp.Variable()  # Investment in municipal bonds
x2 = cp.Variable()  # Investment in bank CDs
x3 = cp.Variable()  # Investment in high-risk account

# Define parameters
total_budget = 12000
high_risk_limit = 2000

# Objective function
objective = cp.Maximize(0.07 * x1 + 0.08 * x2 + 0.12 * x3)

# Constraints
constraints = [
    x1 + x2 + x3 <= total_budget,
    x3 <= high_risk_limit,
    x1 >= 3 * x2
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal Investment Amounts:")
print("Investment in Municipal Bonds :", x1.value)
print("Investment in Bank CDs :", x2.value)
print("Investment in High-Risk Account :", x3.value)


Optimal Investment Amounts:
Investment in Municipal Bonds : 7500.000000326279
Investment in Bank CDs : 2499.999999695155
Investment in High-Risk Account : 1999.999999955671


<font size="8">Example 2</font>

A financial institution has a lending capacity of $1,000,000 and considers three borrower types: A, B, and C with expected returns of 8%, 10%, and 12%, respectively. The institution must allocate loans to all types, with minimums of 30% to A, and 20% to C, and a maximum of 40% to B. The goal is to maximize the total expected return. 

# Loan Allocation Optimization Problem

A financial institution has a lending capacity of $1,000,000 and considers three borrower types: A, B, and C with expected returns of 8%, 10%, and 12%, respectively. The institution must allocate loans to all types, with minimums of 30% to A, and 20% to C, and a maximum of 40% to B. The goal is to maximize the total expected return.

## Objective Function
The objective is to maximize the total expected return:
$ \text{Maximize} \quad 0.08x_A + 0.10x_B + 0.12x_C $

where:
- $x_A$: amount allocated to borrower type A
- $x_B$: amount allocated to borrower type B
- $x_C$: amount allocated to borrower type C

## Constraints
1. **Total loan allocation is limited to $1,000,000:**
   $x_A + x_B + x_C = 1,000,000$

2. **Minimum allocation constraints:**
   - $x_A \geq 0.30 \times 1,000,000$ (30% to A)
   - $x_C \geq 0.20 \times 1,000,000$ (20% to C)

3. **Maximum allocation constraint:**
   - $x_B \leq 0.40 \times 1,000,000$ (40% to B)

## Python Code Using cvxpy

In [23]:
import cvxpy as cp

# Define variables
xA = cp.Variable()  # Loan allocation to borrower type A
xB = cp.Variable()  # Loan allocation to borrower type B
xC = cp.Variable()  # Loan allocation to borrower type C

# Define parameters
total_capacity = 1000000

# Objective function
objective = cp.Maximize(0.08 * xA + 0.10 * xB + 0.12 * xC)

# Constraints
constraints = [
    xA + xB + xC == total_capacity,
    xA >= 0.30 * total_capacity,
    xC >= 0.20 * total_capacity,
    xB <= 0.40 * total_capacity,
    xA >= 0,
    xB >= 0,
    xC >= 0
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal Loan Allocations:")
print("Loan to Borrower A :", xA.value)
print("Loan to Borrower B :", xB.value)
print("Loan to Borrower C :", xC.value)
print("Total Expected Return:", problem.value)


Optimal Loan Allocations:
Loan to Borrower A : 300000.0000006506
Loan to Borrower B : 5.192941456057914e-05
Loan to Borrower C : 699999.9999474199
Total Expected Return: 107999.99999893537


<font size="8">Example 3</font>

A company has a currency exposure of 100,000 units in USD and EUR. It can hedge using Forward Contracts ($0.02 per unit) and Options ($0.03 per unit). The goal is to minimize the total hedging cost while meeting the following constraints:

- Total exposure must be fully hedged.
- Forward Contracts usage cannot exceed 70,000 units.
- At least 30,000 units must be hedged using Options.

# Currency Exposure Hedging Optimization Problem

A company has a currency exposure of 100,000 units in USD and EUR. It can hedge using Forward Contracts ($0.02 per unit) and Options ($0.03 per unit). The goal is to minimize the total hedging cost while meeting the following constraints:

## Objective Function
The objective is to minimize the total hedging cost:
$ \text{Minimize} \quad 0.02x_{\text{Forward}} + 0.03x_{\text{Options}} $

where:
- $x_{\text{Forward}}$: units hedged using Forward Contracts
- $x_{\text{Options}}$: units hedged using Options

## Constraints
1. **Total exposure must be fully hedged:**
   $x_{\text{Forward}} + x_{\text{Options}} = 100,000$

2. **Forward Contracts usage cannot exceed 70,000 units:**
   $x_{\text{Forward}} \leq 70,000$

3. **At least 30,000 units must be hedged using Options:**
   $x_{\text{Options}} \geq 30,000$


In [24]:
# Define variables
x_forward = cp.Variable()  # Units hedged using Forward Contracts
x_options = cp.Variable()  # Units hedged using Options

# Define parameters
forward_cost_per_unit = 0.02
options_cost_per_unit = 0.03
total_exposure = 100000
forward_contract_limit = 70000
options_minimum = 30000

# Objective function
objective = cp.Minimize(forward_cost_per_unit * x_forward + options_cost_per_unit * x_options)

# Constraints
constraints = [
    x_forward + x_options == total_exposure,
    x_forward <= forward_contract_limit,
    x_options >= options_minimum
]

# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display the results
print("Optimal Hedging Strategy:")
print("Units Hedged using Forward:", x_forward.value)
print("Units Hedged using Options:", x_options.value)
print("Total Hedging Cost:", problem.value)

Optimal Hedging Strategy:
Units Hedged using Forward: 69999.99999996346
Units Hedged using Options: 30000.00000003655
Total Hedging Cost: 2300.0000000003656


<font size="8">Quadratic programming</font>

\begin{align*}
\text{minimize:} & (1/2)x^TPx+q^Tx, \\
\text{subject to:} & Gx \leq h, \\
 & Ax = b \\
\end{align*}
\
where

$P \in  \mathbb{R}^{n \times n}, q \in \mathbb{R}^n, G \in \mathbb{R}^{m \times n}, h \in \mathbb{R}^m, A \in \mathbb{R}^{p \times n} \text{ and } b \in \mathbb{R}^n \text{ are problem data and } x \in \mathbb{R}^n \text{ is the optimization variable}$ 

<font size="8">Example 4</font>

## Problem Statement

A company has a total advertising budget of $1,000,000 and wants to allocate this budget across three marketing channels: online advertising, television advertising, and print advertising. The goal is to maximize the overall brand exposure and engagement, which is measured by a quadratic utility function representing the impact of advertising spending on each channel.

The quadratic utility functions for each channel are as follows:

1. **Online Advertising:** $ U_{\text{online}}(x_{\text{online}}) = -0.001x_{\text{online}}^2 + 500x_{\text{online}} $
2. **Television Advertising:** $ U_{\text{TV}}(x_{\text{TV}}) = -0.0005x_{\text{TV}}^2 + 300x_{\text{TV}} $
3. **Print Advertising:** $ U_{\text{print}}(x_{\text{print}}) = -0.0007x_{\text{print}}^2 + 400x_{\text{print}} $

where $x_{\text{online}}$, $x_{\text{TV}}$, and $x_{\text{print}}$ are the amounts spent on online, TV, and print advertising, respectively.

The total budget constraint is:

$ x_{\text{online}} + x_{\text{TV}} + x_{\text{print}} = 1,000,000 $

## Objective Function

The objective is to maximize the overall quadratic utility:

$ \text{Maximize} \quad U(x) = U_{\text{online}}(x_{\text{online}}) + U_{\text{TV}}(x_{\text{TV}}) + U_{\text{print}}(x_{\text{print}}) $

## Constraints

Subject to the total budget constraint:

$ x_{\text{online}} + x_{\text{TV}} + x_{\text{print}} = 1,000,000 $



In [9]:
# Define variables
x_online = cp.Variable()
x_tv = cp.Variable()
x_print = cp.Variable()

# Define parameters
budget = 1000000

# Define quadratic utility functions
U_online = -0.001 * x_online**2 + 500 * x_online
U_tv = -0.0005 * x_tv**2 + 300 * x_tv
U_print = -0.0007 * x_print**2 + 400 * x_print

# Objective function to maximize overall utility
objective = cp.Maximize(U_online + U_tv + U_print)

# Total budget constraint
budget_constraint = [x_online + x_tv + x_print == budget]

# Formulate the problem
problem = cp.Problem(objective, budget_constraint)

# Solve the problem
problem.solve()

# Display the results
print("Optimal solution:")
print("Online Advertising Budget:", x_online.value)
print("Television Advertising Budget:", x_tv.value)
print("Print Advertising Budget:", x_print.value)


Optimal solution:
Online Advertising Budget: 287096.7741928672
Television Advertising Budget: 374193.548386395
Print Advertising Budget: 338709.6774186611


<font size="8">Example 5</font>

## Objective Function (to minimize):
$ f(x, y) = 2x^2 + 5y^2 - 4xy + 6x + 8y $

## Subject to the constraint:
$ 3x + 2y \leq 18 $

In matrix form, the objective function can be written as:
$ f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T Q \mathbf{x} + \mathbf{c}^T \mathbf{x} $

Where:
$ \mathbf{x} = \begin{bmatrix} x \\ y \end{bmatrix} $

$ Q = \begin{bmatrix} 4 & -2 \\ -2 & 10 \end{bmatrix} $

$ \mathbf{c} = \begin{bmatrix} 6 \\ 8 \end{bmatrix} $

And the constraint is:
$ 3x + 2y \leq 18 $



In [10]:
# Define the variables
x = cp.Variable()
y = cp.Variable()

# Define the objective function to minimize
Q = np.array([[4, -2], [-2, 10]])
c = np.array([6, 8])
objective = cp.Minimize(0.5 * cp.quad_form(cp.vstack([x, y]), Q) + c.T @ cp.vstack([x, y]))

# Define the constraint
constraint = [3 * x + 2 * y <= 18]

# Create the optimization problem
problem = cp.Problem(objective, constraint)

# Solve the problem
problem.solve()

# Display the results
print("Optimal x:", x.value)
print("Optimal y:", y.value)
print("Minimum value of the objective function:", problem.value)


Optimal x: -2.1111111111111103
Optimal y: -1.2222222222222225
Minimum value of the objective function: -11.222222222222225


<font size="8">Example 6</font>

Minimize the objective function:

$ f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T Q \mathbf{x} + \mathbf{c}^T \mathbf{x} $

Where:
$ \mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} $

$ Q = \begin{bmatrix} 2 & -1 & 0 \\ -1 & 2 & -1 \\ 0 & -1 & 2 \end{bmatrix} $

$ \mathbf{c} = \begin{bmatrix} -1 \\ -2 \\ -3 \end{bmatrix} $

Subject to the constraints:

$ \begin{bmatrix} 1 & 0 & 1 \\ 0 & 1 & -1 \end{bmatrix} \mathbf{x} \leq \begin{bmatrix} 1 \\ 2 \end{bmatrix} $

$ \begin{bmatrix} 1 & 1 & 0 \end{bmatrix} \mathbf{x} = \begin{bmatrix} 1 \end{bmatrix} $




In [17]:
import cvxpy as cp
import numpy as np

# Define the variables
x = cp.Variable(3)

# Define the objective function
Q = np.array([[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]])

c = np.array([-1, -2, -3])
objective = cp.Minimize(0.5 * cp.quad_form(x, Q) + c.T @ x)

# Define the linear constraints
A = np.array([[1, 0, 1],
              [0, 1, -1]])

b = np.array([1, 2])

# Define equality constraints
A_eq = np.array([[1, 1, 0]])
b_eq = np.array([1])

# Define the problem
constraints = [A @ x <= b, A_eq @ x == b_eq]
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Print the results
print("Optimal value:", problem.value)
print("Optimal x:", x.value)


Optimal value: -4.083333333333334
Optimal x: [-0.16666667  1.16666667  1.16666667]
